In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pymc3 as pm
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

plt.style.use("dark_background")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Normal distribution
We first create some random data from a normal distribution with $\mu=0$ $\sigma=1$ : $ N(0,1) $

In [ ]:
np.random.seed(42)
observed=np.random.randn(100)
sns.distplot(observed);

So we now want to check from what distribution this data could have come from (we pretend that we do not know it). 
Assume that we know $\sigma$ but want to know $\mu$. One very simple way would be to do:

In [ ]:
mean = np.mean(observed)
mean

In [ ]:
x = np.linspace(-2,2,1000)
y = stats.norm.pdf(x = x, loc=0, scale=1)
y_pred = stats.norm.pdf(x = x, loc=mean, scale=1)

fig,ax=plt.subplots()
ax.plot(x,y, label='Actual distribution')
ax.plot(x,y_pred, label='Predicted distribution')

ax.legend();


As an alternative we can use PyMC3 to rather get a probability distribution for $\mu$.

In [ ]:
with pm.Model() as model:
    mu = pm.Uniform("mu", lower=-5, upper=5)
    
    obs = pm.Normal("obs", mu=mu, sigma=1, observed=observed)
    y_pred = pm.Normal("y_pred", mu=mu, sigma=1)  # Omitting the obeserved data will produce a prediction.
    
    trace = pm.sample(10000, random_seed=42, return_inferencedata=False)

In [ ]:
burnin = 500

y_pred = trace[burnin:].get_values('y_pred')
mu = trace[burnin:].get_values('mu')


In [ ]:
fig,ax=plt.subplots()
sns.distplot(y_pred)

fig,ax=plt.subplots()
sns.distplot(mu)
ax.set_title(r'$\mu$')

In [ ]:
with model:
    pm.traceplot(trace);

In [ ]:
with model:
    pm.plot_posterior(trace);